In [ ]:
from __future__ import print_function

import sys
import numpy as np
import math
from time import time

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("./cordic.bit")
    ipDesign = ol.cordic_0
    ipDMAIn0 = ol.axi_dma_0
    ipDMAOut0 = ol.axi_dma_1
    ipDMAOut1 = ol.axi_dma_2

    inBuffer0 = allocate(shape=(89,), dtype=np.uint16)
    outBuffer0 = allocate(shape=(89,), dtype=np.uint16)
    outBuffer1 = allocate(shape=(89,), dtype=np.uint16)
    
    sin = []
    cos = []
 
    for i in range(1, 90):
        radian = i * math.pi / 180
        sin.append(math.sin(radian))
        cos.append(math.cos(radian))
        inBuffer0[i - 1] = radian * (2 ** 14)
       
    timeKernelStart = time()

    ipDesign.write(0x00, 0x01)
    
    ipDMAIn0.sendchannel.transfer(inBuffer0)
    ipDMAOut0.recvchannel.transfer(outBuffer0)
    ipDMAOut1.recvchannel.transfer(outBuffer1)
    ipDMAIn0.sendchannel.wait()
    ipDMAOut0.recvchannel.wait()
    ipDMAOut1.recvchannel.wait()
    
    timeKernelEnd = time()
    
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    diff = 0
    for i in range(89):
        diff += abs(outBuffer0[i] / (2 ** 14) - sin[i])
        diff += abs(outBuffer1[i] / (2 ** 14) - cos[i])
    
    print("difference")
    print(diff)
    
    print("============================")
    print("Exit process")